# Predicting The Office episode ratings to advise NBC Universal on a reunion episode
#### Machine Learning in Python - Group Project 1
*Data Scientist Contractors: Luke Appleby, Josep Reyero, Tom Birkbeck, Adam Dunajski*

In this report, we explore what makes a great episode of The Office, NBC Universal's widely-regarded situational comedy about the office life of a paper manufacturer. 

This is achieved through a machine learning model which predicts the rating of the episode based on a number of episode characteristics - such as whether the episode features a cold open, and how many spoken lines are in it. 

We use this model to give a reccomendation on episode features which will **maximise IMBD rating** for the episode.

The report is formatted in the style of an interactive workbook in which the client (NBC Universal) can step through cells of code interspersed with explanation of model choices and mathematical background.

### Table of Contents

0.  Predicting The Office episode ratings to advise NBC Universal on a reunion episode
1. Introduction    
    1.1 Model and conclusions overview/precis    
    1.2 Data sources used    
    1.3 General code setup       
2. Exploratory Data Analysis and Feature Engineering    
    2.1 **<span style="color:red">SECTION</span>**    
    2.2 **<span style="color:red">SECTION</span>**     
    2.3 **<span style="color:red">SECTION</span>**    
    2.4 **<span style="color:red">SECTION</span>**    
3. Model fitting and tuning    
    3.1 **<span style="color:red">SECTION</span>**    
    3.2 **<span style="color:red">SECTION</span>**    
    3.3 **<span style="color:red">SECTION</span>**    
4. Discussion and Conclusions    
    4.1 **<span style="color:red">SECTION</span>**    
    4.2 **<span style="color:red">SECTION</span>**    
    4.3 **<span style="color:red">SECTION</span>**    
5. References

## 1 Introduction
In this section, we give an abstract of our modelling approach and truncated results, intended for senior managers who are short on time to view. 

We then discuss our data sources and set-up of the code environment for the model.

## 1.1 Model and conclusions overview/precis

After comparing a number of models, we found a **<span style="color:red">[WHICH MODEL DID WE FIND BEST]</span>** model was the best to fit to the data. It had a prediction accuracy of **<span style="color:red">[WHAT WAS ITS ACCURACY]</span>**, which is very high.

The model indicates that strong postive correlants of episodic IMBD rating are **<span style="color:red">[WHAT WERE THE CORRELANTS]</span>**, therefore, we advise NBC Universal to apply the following list of reccomendations when writing the reunion episode.


<span style="color:red">[WHAT WERE THE CORRELANTS]
* COLD OPEN
* OTHER THINGS
    </span>

## 1.2 Data sources
Our primary data source was the office episode ratings dataset the_office.csv (Evkaya, 2023).

## 1.3 General Setup


In [1]:
# Add any additional libraries or submodules below

# Data libraries
import numpy as np
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules that are necessary
import sklearn

In [2]:
# Load data
data = pd.read_csv("the_office.csv")
data_explore = data.copy()

In [3]:
with open('MLP_cold_opens.txt', 'r') as file:
    new_data = file.read().replace('\n', '')

x = new_data.split(")")
hots = []
for i in range(len(x)):
    if x[i][0:4] == 'None':
        hots.append(x[i][7:-1])
    elif x[i][0:8] == 'Season 1' and x[i][8:12] == 'None':
        hots.append(x[i][14:])
    elif x[i][0:6] == 'Season' and x[i][8:12] == 'None':
        hots.append(x[i][15:-1])

hots_df = pd.DataFrame (hots, columns = ['episode_name'])
hots_df['cold_open'] = np.zeros(12).astype(int)
hots_df = hots_df.replace(to_replace = 'Weight Loss', value = 'Weight Loss (Parts 1&2)')
data_ = pd.merge(data, hots_df, on='episode_name', how='left')
data_['cold_open'] = data_['cold_open'].fillna(1)

data_.loc[data_['cold_open'] == 1]

data = data_

In [4]:
with open('MLP_jim_pranks.txt', 'r') as file:
    pranks_str = file.read().replace('\n', '')
    
x0 = pranks_str.split('(')
j = 0
for i in x0:
    x0[j] = i.split(')')
    j+=1
j = 0
for i in x0:
    x0[j] = i[0]
    j +=1
for i in range(len(x0)):
    x0[i] = x0[i][1:-1]


names = data['episode_name']

names = names.replace(to_replace = 'E-Mail Surveilance', value = 'Email Surveillance')
names = names.replace(to_replace = 'A Benihana Christmas ', value = 'A Benihana Christmas')
names = names.replace(to_replace = 'The Job ', value = 'The Job', regex = True)
names = names.replace(to_replace = 'Launch Party ', value = 'Launch Party', regex = True)
names = names.replace(to_replace = 'Goodbye, Toby ', value = 'Goodbye Toby', regex = True)
names = names.replace(to_replace = 'Niagara ', value = 'Niagara', regex = True)
names = names.replace(to_replace = 'The Cover', value = 'The Cover-Up', regex = True)
names = names.replace(to_replace = 'Classy Christmas ', value = 'Classy Christmas', regex = True)
names = names.replace(to_replace = 'Dwight K. Schrute, (Acting) Manager', value = 'Dwight K. Schrute,')
names = np.array(names)

j = 0
for i in names:
    if i[-11:] == '(Parts 1&2)':
        names[j] = i[:-11]
    elif i[-8:] == '(Part 1)' or i[-8:] == '(Part 2)':
        names[j] = i[:-8]
    j +=1
        
pranks = []
for i in names:
    #print(x0.count(i))
    pranks.append(x0.count(i))
#     a += x0.count(i)

data['jim_pranks'] = pranks

data
data_explore = data.copy()

In [5]:
with open('jim_and_pam_key_episodes.txt', 'r') as file:
    text = file.read()
    
start = 0
pam_jim_episodes = []

while True:
    start = text.find('(', start) + 1
    if start == 0:
        break
        
    end = text.find(')', start)
    between_brackets = text[start:end]
    
    if any(char.isdigit() for char in between_brackets):
        season = int(between_brackets[1])
        after_E_string = between_brackets[between_brackets.find('E')+1:]
        
        if '/' in after_E_string:
            pam_jim_episodes.append([season, int(after_E_string[:after_E_string.find('/')])])
            pam_jim_episodes.append([season, int(after_E_string[after_E_string.find('/')+1:])])
        
        else:
            pam_jim_episodes.append([season, int(after_E_string)])
        
    start = end + 1
    
    
def is_key_episode(row, key_episodes):
    season, episode = row['season'], row['episode']
    return int([season, episode] in key_episodes)

data_explore['pam_and_jim_key_episode'] = data_explore.apply(
    lambda row: is_key_episode(row, pam_jim_episodes), axis=1
)

FileNotFoundError: [Errno 2] No such file or directory: 'jim_and_pam_key_episodes.txt'

## 1. Introduction

*This section should include a brief introduction to the task and the data (assume this is a report you are delivering to a client).* 

- If you use any additional data sources, you should introduce them here and discuss why they were included.

- Briefly outline the approaches being used and the conclusions that you are able to draw.

## 2. Exploratory Data Analysis and Feature Engineering

*Include a detailed discussion of the data with a particular emphasis on the features of the data that are relevant for the subsequent modeling.* 

- Including visualizations of the data is strongly encouraged - all code and plots must also be described in the write up. 
- Think carefully about whether each plot needs to be included in your final draft - your report should include figures but they should be as focused and impactful as possible.

*Additionally, this section should also implement and describe any preprocessing / feature engineering of the data.*

- Specifically, this should be any code that you use to generate new columns in the data frame `d`. All of this processing is explicitly meant to occur before we split the data in to training and testing subsets. 
- Processing that will be performed as part of an sklearn pipeline can be mentioned here but should be implemented in the following section.*

**All code and figures should be accompanied by text that provides an overview / context to what is being done or presented.**

In [ ]:
print ("Columns in the df are:")
print (data_explore.columns.tolist(), "\n")

dup_num = pd.DataFrame(data_explore).duplicated().sum()
print("There are {} duplicated values in the data".format(dup_num))

na_num = data_explore.isna().sum().sum()
print(f"There are {na_num} Na values in the data")

data_explore.describe()


Measuring the correlation between the current numerical variables:

In [ ]:
sns.set(rc={'figure.figsize': (10, 6)})
sns.heatmap(data_explore.corr(), annot = True, fmt = '.2f', linewidths = 2)
plt.title("Correlation Heatmap for current numerical variables")
plt.show()

We will first drop the episode name and episode number features because they are not relevant to the model.

In [ ]:
data_explore = data_explore.drop("episode_name", axis=1)
data_explore = data_explore.drop("episode", axis=1)

print (data_explore.columns.tolist())

Next, we will use the variable "air_date" to create two new features: Categorical variable "calendar_season", being the season (autumn, winter, spring, summer) when the episode was aired; and (numerical variable?) "air_month" being the month the episode aired. After this we drop "air_date".

In [ ]:
# data['air_date'] = pd.to_datetime(data['air_date'])

# convert the 'dates' column to datetime format
data_explore['air_date'] = pd.to_datetime(data_explore['air_date'])

# create a new categorical column 'calendar_season' by assigning a category "winter", 
# "spring", "summer", "autumn" depending on the air-month.
data_explore['calendar_season'] = pd.cut(
    data_explore['air_date'].dt.month, 
    [0, 3, 6, 9, 12], 
    labels=['winter', 'spring', 'summer', 'autumn']
)

# convert calender season to string column rather than categorical column.
data_explore['calendar_season'] = data_explore['calendar_season'].astype(str)

# Create month column.
data_explore['month'] = data_explore['air_date'].dt.month

data_explore.head(5)


In [ ]:
data_explore = data_explore.drop("air_date", axis=1)
data_explore.head(5)

In [ ]:
list(data_explore.columns)

We now write a one hot encoder function to assign a binary attribute per category for the Categorical variables ("director, "writer", "calendar_season" and "main_chars"):

In [ ]:
def OneHot_encoder(df, cat_variable):
    oneHot_df = df.copy()
    variable_initials = ''.join([word[0] for word in cat_variable.split("_")])

    
    unique_chars = set(';'.join(oneHot_df[cat_variable]).split(';'))

    # Create binary attributes for each unique character
    for char in unique_chars:
        oneHot_df[variable_initials+"_"+char] = oneHot_df[cat_variable].apply(lambda x: 1 if char in x else 0)
    
    
    # Drop the original main_chars column
    oneHot_df = oneHot_df.drop(columns=[cat_variable])
    
    return oneHot_df


cat_variables = ['director', 'writer', 'main_chars', 'calendar_season']

for cat_variable in cat_variables:
    data_explore = OneHot_encoder(data_explore, cat_variable)
    

data_explore.head(5)


In [ ]:
columns = list(data_explore.columns)
mc_list = [columns[i] for i in range(len(columns)) if columns[i][:3] == "mc_"]
n = len(mc_list)

In [ ]:
pairnames = []
for i in range(n-1):
    for j in range(i+1,n):
        pairname = "pair_" + mc_list[i][3:] + "_"+ mc_list[j][3:]
        pairnames.append(pairname)
        data_explore[pairname] = ((data_explore[mc_list[i]] + data_explore[mc_list[j]])//2)

In [ ]:
corr_mat = data_explore.corr()["imdb_rating"]

corr_array = np.zeros((n,n))
for i in range(n-1):
    for j in range(i+1,n):
        pairname = "pair_" + mc_list[i][3:] + "_"+ mc_list[j][3:]
        corr_array[i][j] = corr_mat[pairname]
        corr_array[j][i] = corr_mat[pairname]
    corr_array[i][i] = corr_mat[mc_list[i]]

In [ ]:
fig, axes = plt.subplots(figsize = (10,10), ncols = 1)

mc_names = [mc_name[3:] for mc_name in mc_list]
plt.grid(b=False)
plt.imshow(corr_array)
plt.colorbar()
plt.xticks(range(n), mc_names, rotation ='vertical')
plt.yticks(range(n), mc_names)
plt.show()

By inspection we realised that there are some repeated directors where in some data entries the name of the director is spelled wrong. The specific cases were:

 - Charles McDougall (correct name) and Charles McDougal
 - Claire Scanlon (correct name) and Claire Scanlong 
 - Greg Daniels (correct name) and Greg Daneils
 - Ken Whittingham (correct name) and Ken Wittingham 
 - Paul Lieberstein (correct name) and Paul Lieerstein
 
Thus we now write code to remove columns with the wrong name, where we will put a 1 in the correct name column if the incorrect one had a 1 and the correct one didn't.

In [ ]:
correct_names = ['d_Charles McDougall','d_Claire Scanlon','d_Greg Daniels', 'd_Ken Whittingham', 'd_Paul Lieberstein']
wrong_names = ['d_Charles McDougal','d_Claire Scanlong','d_Greg Daneils', 'd_Ken Wittingham', 'd_Paul Lieerstein']

for i, correct_name in enumerate(correct_names):
    data_explore.loc[data_explore[wrong_names[i]] == 1, correct_name] = 1
    data_explore = data_explore.drop(wrong_names[i], axis=1)
    
data_explore.head(5)

In [ ]:
list(data_explore.columns)

## 3. Model Fitting and Tuning

*In this section you should detail your choice of model and describe the process used to refine and fit that model.*

- You are strongly encouraged to explore many different modeling methods (e.g. linear regression, regression trees, lasso, etc.) but you should not include a detailed narrative of all of these attempts. 
- At most this section should mention the methods explored and why they were rejected - most of your effort should go into describing the model you are using and your process for tuning and validatin it.

*For example if you considered a linear regression model, a classification tree, and a lasso model and ultimately settled on the linear regression approach then you should mention that other two approaches were tried but do not include any of the code or any in depth discussion of these models beyond why they were rejected. This section should then detail is the development of the linear regression model in terms of features used, interactions considered, and any additional tuning and validation which ultimately led to your final model.* 

**This section should also include the full implementation of your final model, including all necessary validation. As with figures, any included code must also be addressed in the text of the document.**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Having done all the data exploration and feature engineering, we not set our starting dataframe to the 
# engineered dataframe.
data = data_explore

We start by fitting a multiple linear regression, and testing performance. We use a sklearn pipeline to scale and fit the model.  features to have a mean of zero and a standard deviation of one. We will sue standardScaler to ensure that all features have equal importance when fitting a linear regression model, and can improve the stability and convergence of the model during training.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('imdb_rating', axis=1), data['imdb_rating'], 
    test_size=0.3, random_state=42
)

model_pipeline = Pipeline([
    ('regressor', LinearRegression())
])


# Fitting the model
model_pipeline.fit(X_train, y_train)

# Testing the model
y_predict = model_pipeline.predict(X_test)

# Printing performance metrics
print(f"Mean squared error: {mean_squared_error(y_test, y_predict)}")
print(f"R-squared: {r2_score(y_test, y_predict)}")

sns.scatterplot(x=y_test, y=y_predict)
plt.xlabel('Actual IMDB rating')
plt.ylabel('Predicted IMDB rating')
plt.title('Linear regression model performance')
plt.show()

## 4. Discussion and Conclusions


*In this section you should provide a general overview of **your final model**, its **performance**, and **reliability**.* 

- You should discuss what the implications of your model are in terms of the included features, predictive performance, and anything else you think is relevant.

- This should be written with a target audience of a NBC Universal executive who is with the show and university level mathematics but not necessarily someone who has taken a postgraduate statistical modeling course. 

- Your goal should be to convince this audience that your model is both accurate and useful.

- Finally, you should include concrete recommendations on what NBC Universal should do to make their reunion episode a popular as possible.

**Keep in mind that a negative result, i.e. a model that does not work well predictively, but that is well explained and justified in terms of why it failed will likely receive higher marks than a model with strong predictive performance but with poor or incorrect explanations / justifications.**

## 5. References

*In this section, you should present a list of external sources (except the course materials) that you used during the project, if any*

Additional data sources can be cited here, in addition to related python documentations, any other webpage sources that you benefited from


* Evkaya, Ozan. 2023. *The Office CSV Data File*, MATH11205: Machine Learning in Python, The University of Edinburgh.